Ollama Connection

In [ ]:
import ollama

response = ollama.chat(
    model='llama3:latest',
    messages=[{
        'role': 'user',
        'content': 'Reply with exactly this: DIABETAINSIGHT AI ENGINE READY'
    }]
)
print(response['message']['content'])

Load Data

In [ ]:
import pandas as pd
import numpy as np
import json
import time
import warnings
warnings.filterwarnings('ignore')

# Load master data
master = pd.read_csv('..Data\\Processed\\master_health_data.csv')
nppa = pd.read_csv('..Data\\Processednppa_drug_prices.csv')

# Key context numbers for AI
total_diabetics = master['Diabetic_Population_Millions'].sum()
national_avg_prevalence = master['Diabetes_Prevalence_Pct'].mean()
national_avg_income = master['Per_Capita_Income_INR_Thousands'].mean()
national_avg_insurance = master['Health_Insurance_Pct'].mean()
cheapest_drug = nppa['Monthly_Cost_INR'].min()
most_expensive_drug = nppa['Monthly_Cost_INR'].max()

print("Data loaded")
print(f"\nNational Context:")
print(f"  Total diabetics: {total_diabetics:.1f} million")
print(f"  Avg prevalence: {national_avg_prevalence:.1f}%")
print(f"  Avg per capita income: ₹{national_avg_income:.0f}K")
print(f"  Avg insurance coverage: {national_avg_insurance:.1f}%")
print(f"  Drug cost range: ₹{cheapest_drug:.0f} – ₹{most_expensive_drug:.0f}/month")

Prompt engine

In [ ]:
def build_state_prompt(row, national_context):
    
    prompt = f"""
You are a Senior Commercial Analytics Consultant at a top pharma strategy firm in India.
You are analyzing market data for a new Type-2 diabetes drug launching across Indian states.

NATIONAL CONTEXT:
- Total diabetic population in India: {national_context['total_diabetics']:.1f} million
- National average diabetes prevalence: {national_context['avg_prevalence']:.1f}%
- National average per capita income: ₹{national_context['avg_income']:.0f}K/year
- National average health insurance coverage: {national_context['avg_insurance']:.1f}%
- First-line drug cost (Metformin): ₹{national_context['cheapest_drug']:.0f}/month
- Most expensive drug (Insulin): ₹{national_context['costliest_drug']:.0f}/month

STATE DATA:
- State: {row['State']}
- Diabetes Prevalence: {row['Diabetes_Prevalence_Pct']}% (national avg: {national_context['avg_prevalence']:.1f}%)
- Prediabetes Prevalence: {row['Prediabetes_Prevalence_Pct']}%
- Diabetic Population: {row['Diabetic_Population_Millions']:.2f} million
- Total At-Risk Population: {row['Total_At_Risk_Millions']:.2f} million
- Per Capita Income: ₹{row['Per_Capita_Income_INR_Thousands']:.0f}K/year
- Health Insurance Coverage: {row.get('Health_Insurance_Pct', 'N/A')}%
- High Blood Sugar - Women: {row.get('Women_High_Sugar_Pct', 'N/A')}%
- High Blood Sugar - Men: {row.get('Men_High_Sugar_Pct', 'N/A')}%
- Overweight - Women: {row.get('Women_Overweight_Pct', 'N/A')}%
- Overweight - Men: {row.get('Men_Overweight_Pct', 'N/A')}%
- Disease Burden Index: {row['Disease_Burden_Index']:.1f}/100
- Commercial Viability Index: {row['Commercial_Viability_Index']:.1f}/100
- Opportunity Score: {row['Opportunity_Score']:.1f}/100
- Launch Tier: {row['Launch_Tier']}
- National Rank: #{int(row['Rank'])} of 30 states

YOUR TASK:
Analyze this state data and return ONLY a JSON object with exactly these 5 keys.
No text before or after. No markdown. No explanation. Just raw JSON.

{{
  "market_narrative": "2-3 sentences explaining WHY this state has this opportunity score. Use specific numbers. Compare to national averages where relevant.",
  "strategic_recommendation": "2-3 sentences on what the commercial team should specifically DO in this state. Mention timing, focus areas, and channel approach.",
  "key_risk": "1-2 sentences on the single biggest commercial risk in this state.",
  "prediabetes_opportunity": "1 sentence on the prevention market opportunity given the prediabetes prevalence.",
  "board_summary_line": "Exactly one sentence for a CEO board presentation. Must include the opportunity score and one key number."
}}
"""
    return prompt


national_context = {
    'total_diabetics': total_diabetics,
    'avg_prevalence': national_avg_prevalence,
    'avg_income': national_avg_income,
    'avg_insurance': national_avg_insurance,
    'cheapest_drug': cheapest_drug,
    'costliest_drug': most_expensive_drug
}

print("Prompt engine ready")
print(f"Sample prompt size: {len(build_state_prompt(master.iloc[0], national_context))} characters")

The Tamil Nadu test

In [ ]:
import re
import json

def parse_ai_response(raw_response):
    clean = raw_response.strip()
    
    # Remove markdown code blocks
    if '```json' in clean:
        clean = clean.split('```json')[1].split('```')[0]
    elif '```' in clean:
        clean = clean.split('```')[1].split('```')[0]
    
    # Find JSON boundaries
    start = clean.find('{')
    end = clean.rfind('}') + 1
    if start != -1 and end > start:
        clean = clean[start:end]
    
    # Fix trailing commas
    clean = re.sub(r',\s*}', '}', clean)
    clean = re.sub(r',\s*]', ']', clean)
    
    try:
        return json.loads(clean)
    except json.JSONDecodeError:
        # Manual extraction — handles newlines and periods inside values
        result = {}
        fields = ['market_narrative', 'strategic_recommendation',
                  'key_risk', 'prediabetes_opportunity', 'board_summary_line']
        for field in fields:
            # This pattern handles periods, commas inside the string value
            pattern = rf'"{field}"\s*:\s*"((?:[^"\\]|\\.)*)"'
            match = re.search(pattern, clean, re.DOTALL)
            if match:
                result[field] = match.group(1).strip()
            else:
                result[field] = 'Analysis unavailable'
        return result

# Verify on Tamil Nadu first
parsed = parse_ai_response(raw)
print("Parser test on Tamil Nadu:\n")
for key, value in parsed.items():
    print(f"{key}:\n{value}\n")

In [ ]:
results = []
total_states = len(master)

print(f"🚀 Starting AI analysis for all {total_states} states...")

print("="*60)

for idx, row in master.iterrows():
    state_name = row['State']
    rank = int(row['Rank'])
    
    print(f"[{idx+1:02d}/{total_states}] {state_name:<22} (Rank #{rank:02d})...",
          end=' ', flush=True)
    
    try:
        prompt = build_state_prompt(row, national_context)
        
        response = ollama.chat(
            model='llama3:latest',
            messages=[{'role': 'user', 'content': prompt}],
            options={'temperature': 0.3, 'num_predict': 800}
        )
        
        parsed = parse_ai_response(response['message']['content'])
        parsed['State'] = state_name
        parsed['Opportunity_Score'] = row['Opportunity_Score']
        parsed['Launch_Tier'] = row['Launch_Tier']
        parsed['Rank'] = rank
        
        results.append(parsed)
        print("✅")
        time.sleep(1)
        
    except Exception as e:
        print(f"❌ {e}")
        results.append({
            'State': state_name,
            'Opportunity_Score': row['Opportunity_Score'],
            'Launch_Tier': row['Launch_Tier'],
            'Rank': rank,
            'market_narrative': 'Error during analysis',
            'strategic_recommendation': 'Rerun required',
            'key_risk': 'Processing error',
            'prediabetes_opportunity': 'N/A',
            'board_summary_line': 'Analysis incomplete'
        })

print("\n" + "="*60)
print(f"Complete: {len(results)}/{total_states} states processed")

🚀 Starting AI analysis for all 30 states...
⏱️  Estimated time: 8–12 minutes

[01/30] Andhra Pradesh         (Rank #04)... ✅
[02/30] Arunachal Pradesh      (Rank #29)... ✅
[03/30] Assam                  (Rank #21)... ✅
[04/30] Bihar                  (Rank #28)... ✅
[05/30] Chhattisgarh           (Rank #25)... ✅
[06/30] Goa                    (Rank #02)... ✅
[07/30] Gujarat                (Rank #10)... ✅
[08/30] Haryana                (Rank #13)... ✅
[09/30] Himachal Pradesh       (Rank #16)... ✅
[10/30] Jharkhand              (Rank #27)... ✅
[11/30] Karnataka              (Rank #06)... ✅
[12/30] Kerala                 (Rank #03)... ✅
[13/30] Madhya Pradesh         (Rank #24)... ✅
[14/30] Maharashtra            (Rank #08)... ✅
[15/30] Manipur                (Rank #23)... ✅
[16/30] Meghalaya              (Rank #26)... ✅
[17/30] Mizoram                (Rank #17)... ✅
[18/30] Nagaland               (Rank #30)... ✅
[19/30] Odisha                 (Rank #19)... ✅
[20/30] Punjab               

In [ ]:
insights_df = pd.DataFrame(results)

# Merge with master
final_df = master.merge(
    insights_df[['State', 'market_narrative', 'strategic_recommendation',
                 'key_risk', 'prediabetes_opportunity', 'board_summary_line']],
    on='State', how='left'
)

final_df.to_csv('..Data\\Processed\\ai_insights_complete.csv', index=False)
print("Saved: data/processed/ai_insights_complete.csv")
print(f"Shape: {final_df.shape}")

# Preview Tier 1 + 2 insights
priority = final_df[final_df['Launch_Tier'].isin([
    'Tier 1 — Launch Immediately',
    'Tier 2 — Launch in 6 Months'
])].sort_values('Rank')

print("\n" + "="*70)
print("   AI-GENERATED INSIGHTS — PRIORITY STATES")
print("="*70)

for _, row in priority.iterrows():
    print(f"\n{'─'*70}")
    print(f"#{row['Rank']} {row['State'].upper()} | "
          f"Score: {row['Opportunity_Score']:.1f} | {row['Launch_Tier']}")
    print(f"{'─'*70}")
    print(f"📊 {row['market_narrative']}")
    print(f"🎯 {row['strategic_recommendation']}")
    print(f"⚠️  {row['key_risk']}")
    print(f"📋 {row['board_summary_line']}")

Saved: data/processed/ai_insights_complete.csv
Shape: (30, 36)

   AI-GENERATED INSIGHTS — PRIORITY STATES

──────────────────────────────────────────────────────────────────────
#1 TAMIL NADU | Score: 78.7 | Tier 1 — Launch Immediately
──────────────────────────────────────────────────────────────────────
📊 Tamil Nadu's high diabetes prevalence (26.3%) and large diabetic population (20.46 million) create a significant market opportunity, driven by the state's higher per capita income (₹172K/year) and health insurance coverage (66.5%). This is 2x the national average of 12.6% and 1.5x the national average of ₹158K/year.
🎯 Launch the new Type-2 diabetes drug immediately, focusing on Tier 1 cities with high-potential HCPs and a channel approach prioritizing digital marketing to reach the large at-risk population (34.78 million).
⚠️  The single biggest commercial risk is the potential competition from existing glucose-lowering therapies, particularly Metformin, which is widely prescribed 

RESULT SUMMARY

In [ ]:
tier1 = final_df[final_df['Launch_Tier']=='Tier 1 — Launch Immediately']
tier2 = final_df[final_df['Launch_Tier']=='Tier 2 — Launch in 6 Months']
tier3 = final_df[final_df['Launch_Tier']=='Tier 3 — Monitor']
tier4 = final_df[final_df['Launch_Tier']=='Tier 4 — Deprioritize']

exec_prompt = f"""
You are writing the Executive Summary for a pharma commercial strategy report
called DiabetaInsight — an AI-powered market entry analysis for a new Type-2
diabetes drug launching in India.

PROJECT CONTEXT:
- Built by: Siddhant Ranjan, PGDM Business Analytics & Marketing
- Data sources: ICMR-INDIAB 2023 (Lancet), NFHS-5 (MoHFW GoI), 
  NPPA Drug Prices June 2025, RBI State Income Data
- AI engine: Llama 3 (local deployment via Ollama)
- States analyzed: 30

KEY NUMBERS:
- Total Indian diabetic population: {total_diabetics:.1f} million
- National average diabetes prevalence: {national_avg_prevalence:.1f}%
- First-line drug cost: ₹{cheapest_drug:.0f}/month (Metformin)
- Most expensive drug: ₹{most_expensive_drug:.0f}/month (Insulin)

TIER 1 — LAUNCH IMMEDIATELY ({len(tier1)} states):
{tier1[['State','Opportunity_Score','Diabetic_Population_Millions']].to_string(index=False)}
Combined diabetic population: {tier1['Diabetic_Population_Millions'].sum():.1f} million

TIER 2 — LAUNCH IN 6 MONTHS ({len(tier2)} states):
{tier2[['State','Opportunity_Score','Diabetic_Population_Millions']].to_string(index=False)}
Combined diabetic population: {tier2['Diabetic_Population_Millions'].sum():.1f} million

TIER 3 — MONITOR ({len(tier3)} states):
{tier3[['State','Opportunity_Score']].to_string(index=False)}

TIER 4 — DEPRIORITIZE ({len(tier4)} states):
{tier4[['State','Opportunity_Score']].to_string(index=False)}

KEY STRATEGIC FINDINGS:
1. Tamil Nadu ranks #1 (78.7/100) — highest disease burden AND commercial viability combined
2. Goa ranks #2 (63.4) despite only 1.5M population — highest per capita income in India
3. Maharashtra ranks #8 despite being India's financial capital — moderate burden + moderate viability
4. Uttar Pradesh ranks #15 despite 231M population — high disease burden, low commercial viability
5. Rajasthan anomaly: high commercial viability (44.6) but low disease burden (12.7) — prevention market

TASK:
Write a formal Executive Summary in exactly 4 paragraphs:

Paragraph 1: The scale of India's diabetes crisis and why this analysis was needed.
Paragraph 2: Methodology — what data was used and how the opportunity score was built.
Paragraph 3: Key findings — specific states, specific numbers, the surprising insights.
Paragraph 4: Strategic recommendations and call to action for the commercial team.

Rules:
- Formal consulting language
- Every paragraph must contain specific numbers
- No bullet points — flowing prose only
- 280-320 words total
- End with a clear business recommendation
"""

print("🤖 Generating Executive Summary...\n")

exec_response = ollama.chat(
    model='llama3:latest',
    messages=[{'role': 'user', 'content': exec_prompt}],
    options={'temperature': 0.4, 'num_predict': 1500}
)

executive_summary = exec_response['message']['content']

print("="*70)
print("        DIABETAINSIGHT — AI EXECUTIVE SUMMARY")
print("="*70)
print(executive_summary)

# Save
with open('..Data\\Processed\\executive_summary.txt', 'w', encoding='utf-8') as f:
    f.write("DIABETAINSIGHT\n")
    f.write("Pharma Commercial Intelligence System — India Diabetes Market\n")
    f.write("="*70 + "\n\n")
    f.write("EXECUTIVE SUMMARY\n\n")
    f.write(executive_summary)
    f.write("\n\n" + "="*70)
    f.write("\nData Sources: ICMR-INDIAB 2023 | NFHS-5 | NPPA June 2025 | RBI")
    f.write("\nAI Engine: Llama 3 (local) | Built by: Siddhant Ranjan")

print("\nExecutive summary saved")

🤖 Generating Executive Summary...

        DIABETAINSIGHT — AI EXECUTIVE SUMMARY
As the Executive Summary for DiabetaInsight, I am pleased to present the findings of our AI-powered market entry analysis for a new Type-2 diabetes drug launching in India. The scale of India's diabetes crisis is staggering, with a national average prevalence rate of 12.6% and a total diabetic population of 165.8 million. This presents a significant commercial opportunity for pharmaceutical companies looking to enter the Indian market.

Our methodology involved leveraging a range of data sources, including ICMR-INDIAB 2023 (Lancet), NFHS-5 (MoHFW GoI), NPPA Drug Prices June 2025, and RBI State Income Data. We utilized Llama 3, an AI engine deployed via Ollama, to analyze the data and generate an opportunity score for each of India's 30 states. This score takes into account both the disease burden and commercial viability in each state.

Our key findings reveal that Tamil Nadu ranks #1 with an opportunity s